In [206]:
#!/usr/bin/env python
import os
from witb.utils import ioutils as io
from witb.globals import DATA_DIR
import time  
import nltk
import csv
import re, string
import collections
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime

In [2]:
DATA_DIR="/home/sasha/Documents/MilaPostDoc/Python/CommonCrawl"

In [292]:
blacklistcsv='../wordlists/porn_blacklist_filtered.csv'

In [87]:
bigrams='/home/sasha/Documents/MilaPostDoc/Python/porn_ngram_filter/wordlists/porn_bigrams.csv'

In [147]:
bigramlist = []
with open(bigrams, newline='') as inputfile:
    for row in csv.reader(inputfile):
        bigramlist.append(tuple(row[0].split()))
print(len(bigramlist))

882


In [148]:
bigramlist

[('hairy', 'pussy'),
 ('wet', 'pussy'),
 ('black', 'pussy'),
 ('tight', 'pussy'),
 ('pussy', 'porn'),
 ('sex', 'videos'),
 ('pussy', 'sex'),
 ('shaved', 'pussy'),
 ('girl', 'pussy'),
 ('pussy', 'pics'),
 ('eating', 'pussy'),
 ('pussy', 'licking'),
 ('her', 'pussy'),
 ('pussy', 'free'),
 ('licking', 'pussy'),
 ('pink', 'pussy'),
 ('teen', 'pussy'),
 ('big', 'pussy'),
 ('pussy', 'fuck'),
 ('eat', 'pussy'),
 ('my', 'pussy'),
 ('naked', 'pussy'),
 ('fat', 'pussy'),
 ('pussy', 'and'),
 ('first', 'time'),
 ('girl', 'sex'),
 ('having', 'sex'),
 ('lesbian', 'pussy'),
 ('porn', 'movies'),
 ('fucking', 'pussy'),
 ('pussy', 'videos'),
 ('sex', 'pussy'),
 ('pussy', 'video'),
 ('skinny', 'teen'),
 ('girl', 'porn'),
 ('in', 'pussy'),
 ('porn', 'videos'),
 ('white', 'pussy'),
 ('free', 'porn'),
 ('sex', 'video'),
 ('and', 'pussy'),
 ('sex', 'with'),
 ('anal', 'sex'),
 ('sex', 'movies'),
 ('pussy', 'pictures'),
 ('little', 'pussy'),
 ('pussy', 'lips'),
 ('click', 'here'),
 ('to', 'watch'),
 ('girls', 

In [17]:
blacklist = []
with open(blacklistcsv, newline='') as inputfile:
    for row in csv.reader(inputfile):
        blacklist.append(row[0])
print(len(blacklist))

3811


In [3]:
filenames = os.listdir(DATA_DIR)
filename = os.path.join(DATA_DIR, filenames[0])

data = io.get_entries(io.read_wet_file(filename))
parsed_data = io.parse_data(data)  # Generator.

docs = []
start_time = time.time()
for doc in parsed_data:
    docs.append(doc)

In [289]:
for doc in docs:
    if doc.url=='macryu.com':
        testdoc=doc.content

In [290]:
testdoc

"iFixit tears down the Apple Pencil → - MacRyu - The Worst Mac Blog in Singapore, and Some Say the World at MacRyu – The Worst Mac Blog in Singapore, and Some Say the World\nMac News Updates\nWhat is this—a logic board for ants? Not quite, but weighing in at a whopping 1.0 gram it’s definitely the smallest we’ve ever seen.\nThis thing has a freaking logic board (with 4 chips on it) inside! Mind blown.\nThe pencil is clearly not meant to be opened or repaired; you can’t get inside without destroying the device.\nDuh. What else did they expect?\nWhile the teardown’s informative and stuff, there is one less Apple Pencil in the world for someone who really wants one urgently. Maybe they could have waited until stock levels become normal.\nPermalink\nShare this article:\nClick to share on Facebook (Opens in new window)\nClick to share on Twitter (Opens in new window)\nClick to share on Reddit (Opens in new window)\nClick to share on Tumblr (Opens in new window)\nClick to share on LinkedIn (

In [160]:
test= str(docs[15].content)

In [167]:
#Simple word search
start = time.time()
pornsites={}
for doc in docs:
    pwds=len([w for w in word_tokenize(doc.content) if w in blacklist])
    pornsites[doc.url]=pwds
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

KeyboardInterrupt: 

In [286]:
#Bigram search
stop_words = set(stopwords.words('english'))  
start = time.time()
pornsites={}
for doc in docs:
    clean = [w.strip(string.digits + string.punctuation).lower() for w in word_tokenize(doc.content) if w not in stop_words]
    bigrams=[b for b in nltk.bigrams(word_tokenize(str(' '.join(clean))))]
    match = set(bigrams).intersection(bigramlist)
    pornsites[doc.url]=len(match)
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

Number of pages counted:  12903
Duration:  0:04:16.810443


In [288]:
{k: v for k, v in sorted(pornsites.items(), key=lambda item: item[1], reverse=True)}

{'www.pornhavana.com': 108,
 'pinoyplushd.com': 77,
 'www.allviewfunding.com': 69,
 'www.fapmovs.tv': 62,
 'musicofporn.relayblog.com': 53,
 'www.sunindianporn.com': 52,
 'prn247.com': 45,
 'verifiedspace.com': 45,
 'annaazizah.ilearning.me': 44,
 'www.shinyakushiji.or.jp': 43,
 'cumshot.zone': 42,
 'hotporn.sex': 41,
 'indianporn4k.com': 41,
 '2ksex.com': 40,
 'dev.radiantnepal.com': 38,
 'hoetoob.com': 35,
 'pornokim.com': 32,
 'xxxvidso.com': 32,
 'ebonyporn.site': 30,
 'wellnesscarefoundation.com': 29,
 'www.momtubexxx.com': 28,
 'www.teenporntube.xxx': 28,
 'penisextendersonreview.com': 28,
 'family-tube.hq-prn.com': 27,
 'www.mofoscasting.com': 26,
 'allfreegay.com': 25,
 'originalhindiporn.mobi': 25,
 'www.vintagporntube.com': 25,
 'matureporn.watch': 24,
 'pornok7.com': 23,
 'stillwissen.com': 23,
 'teenpussysex.net': 23,
 'topincestvideos.com': 23,
 'www.flashonthebeach.com': 22,
 'sexyvintage80s.com': 22,
 'www.xn--80aafblbgpxxcgbigyfoeei.xn--p1ai': 21,
 'freesexgames.cc': 21